In [13]:
import pandas as pd
from uszipcode import SearchEngine

search = SearchEngine(SearchEngine.SimpleOrComprehensiveArgEnum.comprehensive)

In [17]:
from tqdm.auto import tqdm
tqdm.pandas()

In [14]:
citystate = {
    'Chicago': 'IL',
    'Denver': 'CO',
    'Des Moines': 'IA',
}

In [15]:
categories = ['bike_rental', 'parking', 'public_transport', 'schools', 'shop']
dfs = dict()

for category in categories:
    dfs[category] = pd.read_csv(f'./data/{category}.csv')

In [38]:
for cat in categories:
    if 'Unnamed: 0' in dfs[cat].columns:
        dfs[cat].drop(columns=['Unnamed: 0'], inplace=True)
    dfs[cat] = dfs[cat].dropna().astype({'Zipcode': int})

In [23]:
def is_correct_city(row):
    if row['City'] not in citystate.keys():
        return False
    return search.by_zipcode(row['Zipcode']).state == citystate[row['City']]

In [39]:
for cat in categories:
    print(f'{cat}')
    dfs[cat]['is_valid'] = dfs[cat].progress_apply(is_correct_city, axis=1)

bike_rental


  0%|          | 0/1752 [00:00<?, ?it/s]

parking


  0%|          | 0/312 [00:00<?, ?it/s]

public_transport


  0%|          | 0/36790 [00:00<?, ?it/s]

schools


  0%|          | 0/1094 [00:00<?, ?it/s]

shop


  0%|          | 0/9896 [00:00<?, ?it/s]

In [42]:
for cat in categories:
    print(cat)
    display(dfs[cat].groupby('is_valid').count())

bike_rental


,City,Lat,Lon,Zipcode
is_valid,,,,
True,1752,1752,1752,1752


parking


,City,Lat,Lon,Zipcode
is_valid,,,,
False,12,12,12,12
True,300,300,300,300


public_transport


,City,Lat,Lon,Zipcode
is_valid,,,,
False,366,366,366,366
True,36424,36424,36424,36424


schools


,City,Lat,Lon,Zipcode
is_valid,,,,
False,14,14,14,14
True,1080,1080,1080,1080


shop


,City,Lat,Lon,Zipcode
is_valid,,,,
False,132,132,132,132
True,9764,9764,9764,9764


In [44]:
for cat in categories:
    print(cat)
    dfs[cat].drop(dfs[cat][dfs[cat]['is_valid'] == False].index, inplace=True)
    dfs[cat].drop(columns=['is_valid'], inplace=True)

bike_rental
parking
public_transport
schools
shop


In [58]:
main_df = pd.DataFrame()
for cat in categories:
    main_df[cat] = dfs[cat].groupby('Zipcode').size()

In [64]:
main_df.fillna(0, inplace=True)
main_df = main_df.astype(int)
main_df.head()

,bike_rental,parking,public_transport,schools,shop
Zipcode,,,,,
50309,24,2,196,0,100
50311,8,0,46,0,34
50312,2,2,52,0,44
50314,4,0,92,0,6
50315,2,0,2,4,28


In [70]:
main_df.describe()

,bike_rental,parking,public_transport,schools,shop
count,73.000000,73.000000,73.000000,73.000000,73.000000
mean,24.000000,1.835616,454.712329,12.356164,103.671233
std,18.260461,3.940578,375.457701,12.637433,112.703650
min,2.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,0.000000,174.000000,2.000000,40.000000
50%,20.000000,0.000000,354.000000,8.000000,66.000000
75%,36.000000,2.000000,612.000000,20.000000,118.000000
max,68.000000,20.000000,1650.000000,46.000000,594.000000


In [66]:
main_df.to_csv('./data/processed/main.csv')

In [69]:
for cat in categories:
    dfs[cat].to_csv(f'./data/processed/{cat}.csv', index=False)